In [ ]:
from ultralytics import YOLO

In [ ]:
!yolo task=detect mode=predict model=../.runs/weights/yolov8l.pt conf=0.25 source="D:/NWR27/MY_FILES/Datasets/z_arc/Murtaza/Images/People.jpg" project="../.runs/detect/.try" name="predict"

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))